In [ ]:
from misc import load, load_class_labels

dataset_root = "./dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train_info_cleaned.csv', 1, sample_rate)
test_info = load(dataset_root + 'labels/test.csv')
val_info = load(dataset_root + 'labels/val_info_cleaned.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values
test_image_names = test_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values
test_labels = test_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

In [ ]:
from cnn_classification import build_finetune_network, train_network
from tensorflow import keras
from keras.applications.efficientnet_v2 import EfficientNetV2B0, preprocess_input

extension = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(251, activation='softmax')])

loss = 'categorical_crossentropy'
optimizer = 'adam'
metrics = ['accuracy']

cut_layer = 'top_dropout'
non_trainable_cut_layer = 'block5e_add'

extended_model = build_finetune_network(EfficientNetV2B0, extension, cut_layer,
                                        optimizer, metrics, loss, non_trainable_cut_layer=non_trainable_cut_layer)
#train_network(extended_model, train_info, val_info, train_dir, epochs=10, augment=True, batch_size=64)

In [ ]:
EfficientNetV2B0().summary()

In [ ]:
extended_model.summary()

In [ ]:
extended_model.save('./finetune_network/model')
from tensorflow import keras
keras.applications.EfficientNetV2B0().summary()

In [ ]:
from tensorflow import keras
from cnn_classification import train_network
import gc

path = './efficientnet_noise'
for i in range(20):
    extended_model = keras.models.load_model(path)
    train_network(extended_model, train_info, val_info, train_dir, epochs=1, augment=True, batch_size=64, preprocess_input=preprocess_input)
    extended_model.save(path)
    del extended_model
    gc.collect()
    keras.backend.clear_session()

In [ ]:
from tensorflow import keras
from cnn_classification import evaluate_model, predict

model = keras.models.load_model(path)
predictions = predict(model, test_info, val_degraded_dir, 64, preprocess_input=preprocess_input)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

num_pred = []
for p in predictions:
    num_pred.append(np.argmax(p))
print(len(predictions), len(test_labels))
lab = np.array(test_labels)[0:11968]
cm = confusion_matrix(lab, num_pred)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()

In [ ]:
cm

In [ ]:
cm.diagonal().sum() / cm.sum()